# Air Quality Visual Map

## Introduction
This documentation provides instructions on how to access and use the Air Quality Visual Map API. The API allows you to retrieve air quality data for various locations and display it on a visual map.

## Understanding Air Quality Index (AQI)
The Air Quality Index (AQI) is a standardized index used to communicate the level of air pollution to the public. It provides valuable information about the quality of the air in a specific location. The AQI is typically divided into different categories, each representing a range of air quality conditions.

AQI Categories:
1. **Good (0-50)**: Air quality is satisfactory, and poses little or no risk to health.
2. **Moderate (51-100)**: Air quality is acceptable; however, some pollutants may be a concern for a very small number of people who are unusually sensitive to air pollution.
3. **Unhealthy for Sensitive Groups (101-150)**: Members of sensitive groups (e.g., children, elderly, individuals with respiratory or heart conditions) may experience health effects, but the general public is unlikely to be affected.
4. **Unhealthy (151-200)**: Everyone may begin to experience health effects; members of sensitive groups may experience more serious health effects.
5. **Very Unhealthy (201-300)**: Health alert: everyone may experience more serious health effects.
Hazardous (301-500): Health warning of emergency conditions; the entire population is likely to be affected.

## Obtaining an API Key
Before you can start using the Air Quality Visual Map API, you need to obtain an API key. Follow these steps to get your API key:
1. Visit the [AQICN Gaia API page](https://aqicn.org/gaia/).
2. Register on the website using your email address and name.
3. After registration, an access token will be sent to your email address almost immediately.
4. Retrieve the access token from your email. This token is your key to access the API.

## Using the API Key
Once you have your API key, you can use it to make requests to the Air Quality Visual Map API and access air quality data. Be sure to keep your API key secure and do not share it publicly.

Here's an example of how to include your API key in your API requests:

```http
GET https://api.waqi.info/feed/<CITY>/?token=YOUR_API_KEY
```

## Step 1: Fetch the Data
In this step, you will use your API key to make requests to the Air Quality Visual Map API and retrieve air quality data for the desired location (replace <CITY> with the name of the city you want data for).

In [610]:
import pandas as pd
import os
TOKEN = os.environ['API_KEY']

#coordinates = "49.5,2.5,51.5,6.5" # Belgium
coordinates = "-11.0,94.0,6.0,141.0" # Indonesia

# Split the latlngbox into min and max values
lat_min, lon_min, lat_max, lon_max = map(float, coordinates.split(','))

# Calculate the center of the bounding box
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

URL = "https://api.waqi.info/map/bounds/?latlng={}&token={}".format(coordinates, TOKEN)

result = pd.read_json(URL)

rows = []
for row in result['data']:
    col = [row['station']['name'], row['lon'], row['lat'], row['aqi']]
    rows.append(col)
    
df = pd.DataFrame(rows, columns=['station_name', 'lon', 'lat', 'aqi'])

print(df.columns)

Index(['station_name', 'lon', 'lat', 'aqi'], dtype='object')


## Step 2: Clean the Data
After fetching the data, it's essential to clean it to ensure that it's in a suitable format for analysis. In this step, we focus on cleaning the 'aqi' column by handling missing values. Any NaN (Not a Number) values in the 'aqi' column will be removed from the dataset, ensuring that the air quality index (AQI) data is free from missing or invalid entries.

In [611]:
# Convert 'aqi' to numeric, invalid parsing becomes NaN
df.aqi = pd.to_numeric(df.aqi, errors='coerce')

# Print the shape of DataFrame with NaN values
print('with NaN ->', df.shape)

# Remove rows with NaN values in the 'aqi' column
df1 = df.dropna(subset=['aqi'])

# Print the shape of DataFrame without NaN values
print('without NaN ->', df1.shape)

with NaN -> (85, 4)
without NaN -> (83, 4)


## Step 3: Plotting the Heat Map
Once the data is cleaned and prepared, you can visualize it on a heat map. A heat map provides a visual representation of air quality data, allowing you to identify areas with different air quality levels.

In [612]:
import folium
from folium.plugins import HeatMap

# Create a Folium map with an initial location
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Create a HeatMap layer with your data
df2 = df1[['lat', 'lon', 'aqi']]

heat_map = HeatMap(df2, radius=20)

# Add the HeatMap layer to the map
m.add_child(heat_map)

# Save the map
m.save("docs/heat-map.html")

# Display the map
m

## Step 5: Plotting the Stations
In addition to the heat map, you can also plot the air quality monitoring stations on the map to provide more detailed information about specific locations.

In [613]:
# Create a second map
m2 = folium.Map(location=[center_lat, center_lon], tiles='Stamen Terrain', zoom_start=5)

# Iterate over the list of dictionaries in result['data']
for row in result['data']:
    lat = row['lat']
    lon = row['lon']
    aqi = row['aqi']

    if aqi.isdigit() and int(aqi) > 300:
        color = 'maroon'
    elif aqi.isdigit() and int(aqi) > 200:
        color = 'purple'
    elif aqi.isdigit() and int(aqi) > 150:
        color = 'red'
    elif aqi.isdigit() and int(aqi) > 100:
        color = 'orange'
    elif aqi.isdigit() and int(aqi) > 50:
        color = 'orange'
    else:
        color = 'green'

    # Create a marker for each data point and add it to m2
    folium.Marker(location=[lat, lon], popup=f"Station: {row['station']['name']} AQI={aqi}",
                  icon=folium.Icon(color=color)).add_to(m2)

# Save the map
m2.save("docs/plotting-stations.html")

# Display map
m2